In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sb
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import balanced_accuracy_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import MinMaxScaler,StandardScaler
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv("/kaggle/input/hell-week-mic/train.csv")
print(data.info())
pd.set_option('display.max_columns', None)
data.describe(include='all')

In [ ]:
data.head()

In [ ]:
data.sample(20)

In [ ]:
data.columns

Dropping unnecessary rows from our train set:

In [ ]:
data1 = data.drop(['UID','ph_no', 'cvv','credit_card_number','job','email','url','country','emoji','name'],axis = 1)
data1.describe()

function to use regex to extract the floats from no- null values

In [ ]:
import re
def extract_float(text):
    if pd.notna(text):
        float_pattern = r'[-+]?\d+\.\d+'
        floats = re.findall(float_pattern, text)
        if floats:
            return float(floats[0])
        else:
            return None
    return None
data2 = data1.drop('state', axis = 1)

In [ ]:
data1.head()

In [ ]:
for i in data2.columns:
    data2[i] = data2[i].map(extract_float)

Clean train data:

In [ ]:
data2.describe()

Encoding state and training model(using n_estimators  as 250 for submission 2):

In [ ]:
X_full = data2.copy()
le1 = preprocessing.LabelEncoder()
y_full = pd.DataFrame()
y_full['state'] = le1.fit_transform(data1['state'])
xgb_model = xgb.XGBClassifier(objective = 'multi:softmax', num_class = 100, max_depth= 8, learning_rate = 0.55, n_estimators = 250)
xgb_model.fit(X_full,y_full, verbose = 1)

Getting test set

In [ ]:
test = pd.read_csv("/kaggle/input/hell-week-mic/test.csv")
print(test.info())
test.describe(include='all')

Cleaning test set:

In [ ]:
test1 = test.drop(['UID','ph_no', 'cvv','credit_card_number','job','email','url','country','emoji','name'],axis = 1)
for i in test1.columns:
    test1[i] = test1[i].map(extract_float)

Making predictions and creating csv:

In [ ]:
pred = xgb_model.predict(test1)
pred1 = le1.inverse_transform(pred)
out = test[['UID']]
out['state'] = pred1
out.to_csv('sub.csv',index=False)